In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


In [2]:
from openai import AzureOpenAI
import os

api_base = 'https://gpt-4-turbo-vision.openai.azure.com/' # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
api_key="3bd8e4e1a2db439e9a499d1690f6e232"
deployment_name = 'gpt-4-vision-preview'
api_version = '2023-12-01-preview' # this might change in the future

In [3]:
client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    base_url=f"{api_base}openai/deployments/{deployment_name}/extensions",
)

In [4]:
import base64
from mimetypes import guess_type

In [5]:
def encode_image(image_path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode('utf-8')

In [6]:
def image_ingredients_azure(imag_base64):
  prompt = "You are a bot that is good at analyzing images. provide list of ingredients present in image"
  response = client.chat.completions.create(
      model=deployment_name,
      messages=[
          { "role": "system", "content": "You are a helpful assistant." },
          { "role": "user", "content": [
              {
                  "type": "text",
                  "text":  prompt,
              },
              {
                  "type": "image_url",
                  "image_url": {"url":f"data:image/png;base64,{imag_base64}"},
              }
              ]
          }
      ],
      max_tokens=2000
    )
  return response.choices[0].message.content

In [8]:
image_ingre = []
for i in os.listdir("./images"):
    if i.endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join("./images", i)
        encoded_image = encode_image(image_path)
        response = image_ingredients_azure(encoded_image)
        image_ingre.append(response)

image_ingre

['The image shows a label listing the ingredients of a food product. Here are the ingredients listed:\n\nNoodles:\n- Wheat flour\n- Edible vegetable oil\n- Salt\n- Wheat gluten\n- Mineral (170(i)) and Guar gum\n\nMasala TASTEMAKER®:\n- Hydrolysed groundnut protein\n- Mixed spices (24.9%) of which (Onion powder, Chilli powder, Garlic powder, Coriander, Turmeric, Cumin, Aniseed, Black pepper, Fenugreek, Ginger, Clove, Nutmeg and Cardamom)\n- Noodle powder (Wheat flour, Edible vegetable oil, Salt, Wheat gluten, Mineral (170(i)) and Guar gum)\n- Sugar\n- Edible starch\n- Edible vegetable oil\n- Salt\n- Acidifying agent (330)\n- Mineral (508)\n- Flavour enhancer (635)\n- Colour (150d) and\n- Raising agent (500(ii))\n\nThe label also states "CONTAINS PERMITTED NATURAL COLOUR."',
 'The ingredients listed in the image are:\n\n- Water\n- Soybean Oil\n- Modified Food Starch\n- Vinegar\n- Sugar\n- Maltodextrin\n- Salt\n- Natural Flavor\n- Eggs\n- Mustard Flour\n- Lactic Acid\n- Potassium Sorbate 